Steps:
- prepare data
- process data
- build the CNN
- compile the model
- train the model

In [ ]:
#import everything important (might have to download first)
import Pkg;
# Pkg.add("LuxAMDGPU")
# Pkg.add("LuxCUDA")
# Pkg.add("JLD2")
# Pkg.add("MLUtils")
# Pkg.add("Optimisers")
# Pkg.add("Zygote")
# Pkg.add("Random")
# Pkg.add("Statistics")
# Pkg.add("Images")
# Pkg.add("FileIO")
# Pkg.add("IterTools")
# Pkg.add("OneHotArrays")
using Lux, LuxAMDGPU, CUDA, JLD2, MLUtils, Optimisers, Zygote, Random, Statistics, Images, FileIO, IterTools, OneHotArray

In [ ]:
#image processing (assumes data is split into 'good' and 'defective' folders)
using Images
using Statistics

global good_path = "C:\\Users\\Vincent Alexander\\OneDrive\\Desktop\\good" #set as global to be accessed in train function
global defective_path = "C:\\Users\\Vincent Alexander\\OneDrive\\Desktop\\defective"

function imgProcess(dir_path::AbstractString, output_path::AbstractString, width::Int, height::Int)
    for filename in readdir(dir_path)
        if endswith(filename, ".jpg")
            img_path = joinpath(dir_path, filename)
            img = load(img_path)
            #resize image
            resizedImage = imresize(img, width, height)
            #convert to array to normalize
            arr = Array(resizedImage)
            mean_val = mean(arr)
            std_val = std(arr)
            normalized_image = (arr .- mean_val) / std_val
            #grayscale the image
            processed_image = Gray.(normalized_image)
            #save processed image in new path
            save(joinpath(output_path, filename), processed_image)
        end
    end
end

imgProcess(good_path, "processed_good_path", 224, 224)
imgProcess(defective_path, "processed_defective_path", 224, 224)


In [2]:
#Collects tire data and splits into batches for training/testing
using Random
using OneHotArrays
using Images
using MLUtils
using Statistics

function load_data(good_tire_path::String, defective_tire_path::String, batchsize::Int64, train_split::Float64)
    x_data = []
    y_data = []
    #put good tires and corresponding labels into x/y data
    for (i, file) in enumerate(readdir(good_tire_path))
        img = load(joinpath(good_tire_path, file))
        push!(x_data, img)
        push!(y_data, 0) #Assign 0 label for good tires
    end
    #put defective tires and corresponding labels into x/y data
    for (i, file) in enumerate(readdir(defective_tire_path))
        img = load(joinpath(defective_tire_path, file))
        push!(x_data, img)
        push!(y_data, 1) #Assign 1 label for defective tires
    end
    #stack images along fourth dimension
    x_data = cat(x_data...; dims=4)
    #shuffle data
    N = length(x_data[1, 1, 1, :])
    num_train = Int(floor(N * train_split))
    indicies = shuffle(1:N)
    
    #split data into training and testing sets
    x_train = x_data[:, :, :, indicies[1:num_train]]
    x_test = x_data[:, :, :, indicies[(num_train + 1):end]]

    y_data = onehotbatch(y_data, 0:1)
    y_train = y_data[:, indicies[1:num_train]]
    y_test = y_data[:, indicies[(num_train+1):end]]
    
    train_loader = MLUtils.DataLoader((x_train, y_train), batchsize)
    test_loader = MLUtils.DataLoader((x_test, y_test), batchsize)
    return train_loader, test_loader
end




load_data (generic function with 1 method)

In [3]:
#establish the model

#define model using chain of layers
model = Chain(
   Conv((3, 3), 3 => 16, relu),
   MaxPool((2, 2)),
   Conv((3, 3), 16 => 32, relu),
   MaxPool((2, 2)),
   GlobalMeanPool(),
   Dense(32, 128, relu),  # Adjust the input size (32) and output size (128)
   Dense(128, 2, sigmoid),  # 2 classes (good or bad)
)

Chain(
    layer_1 = Conv((3, 3), 3 => 16, relu),  # 448 parameters
    layer_2 = MaxPool((2, 2)),
    layer_3 = Conv((3, 3), 16 => 32, relu),  # 4_640 parameters
    layer_4 = MaxPool((2, 2)),
    layer_5 = GlobalMeanPool(),
    layer_6 = Dense(32 => 128, relu),   # 4_224 parameters
    layer_7 = Dense(128 => 2, sigmoid_fast),  # 258 parameters
)         # Total: 9_570 parameters,
          #        plus 0 states.

In [4]:
#define the loss function
function xlogy(x, y)
    result = x * log(y)
    return ifelse(iszero(x), zero(result), result)
end

function binarycrossentropy(y_pred, y_true)
    y_pred = y_pred .+ eps(eltype(y_pred))
    return mean(@. -xlogy(y_true, y_pred) - xlogy(1 - y_true, 1 - y_pred))
end

function compute_loss(x, y, model, ps, st)
    y_pred, st = model(x, ps, st)
    return binarycrossentropy(y_pred, y), y_pred, st
end

matches(y_pred, y_true) = sum((y_pred .> 0.5) .== y_true)
accuracy(y_pred, y_true) = matches(y_pred, y_true) / length(y_pred)

accuracy (generic function with 1 method)

In [5]:
#define optimiser
function create_optimiser(ps)
    opt = Optimisers.ADAM(0.01f0)
    return Optimisers.setup(opt, ps)
end

create_optimiser (generic function with 1 method)

In [ ]:
#train the model
batchsize = 100
train_split = 0.6
train_loader, test_loader = load_data("processed_good_path", "processed_defective_path", batchsize, train_split)

ps = params(model)

optimiser = create_optimiser(ps)
init!(optimiser, ps)

epochs = 10

function train(train_loader, test_loader, model, optimiser, ps, epochs)
    for epoch in 1:epochs
        total_loss = 0.0
    
        for (x, y) in train_loader
            grads, loss = gradient(ps) do
                y_pred = model(x, ps)
                loss, y_pred = compute_loss(x, y, model, ps, nothing )
            end
            total_loss += loss
            update!(optimizer, ps, grads)
        end
    
        println("Epoch: $epoch, Loss: $total_loss")
    end
end

train(train_loader, test_loader, model, optimiser, ps, epochs)